#  Texture Encoder


### **Purpose:** 
The purpose of this texture encoder is to create a many to one distillation of images using neural networks.  There are 2 modes to this. One is a residual network that adds a mask to the original image and calculates loss based on the new image, and the other is a complete reconstruction of the image without a residual component to the network. The goal is to create a mapping between many textures to one common texture to aid a reinforcement learning network in simulation to enable adaptability in transfering learning from one texture to that of another.  All of the input images to this network are images captured from random coorinates on a proceduraly generated maze from the Unity Game Engine.

### **Setup:** 
A requirement for this program to work is to have many folder in this directory that contains many folders all with the same images, but in different textures, with one folder named "real".  The "real" folder will be the texture that the other textures will map to.


#### Configure arguments for this network:
`parser = argparse.ArgumentParser()
parser.add_argument('--dataset_dir', dest='dataset_dir', default='data', help='path of the directories containing textured images')
parser.add_argument('--generalized_data', dest='generalized_data', default='real', help='path within dataset_dir that contains the images to be generalized to')
parser.add_argument('--test_dir_x', dest='test_dir_x', default='cropped_10k_lsun_test', help='path of the test dataset')
parser.add_argument('--ck_path', dest='ck_path', default='checkpoints', help='checkpoint path')
parser.add_argument('--epoch', dest='epoch', type=int, default=200000, help='# of epoch')
parser.add_argument('--learning_rate', dest='lr', type=float, default=.0001,help='learning rate')
parser.add_argument('--image_shape', dest='image_shape',default=[256,256,3], help='shape of each image')
parser.add_argument('--vector_dims', dest='vector_dims',default=1000, help='Dimensionality of the encoded vector')
parser.add_argument('--small_network', dest='small_network',default=False, help='use simple network')
parser.add_argument('--residual', dest='residual',default=False, help='use a residual network')
parser.add_argument('--load', dest='load',default=True, help='Load weights')
args = parser.parse_args()`

#### Instantiate and train Phi (texture transfer function)

In [ ]:
    with tf.Session(config=tfconfig) as sess:
        model = phi(sess, args)
        model.train(args)

   ### Initialize values for Phi with specified Arguments
    
    def __init__(self, sess, args):
        self.sess = sess
        self.dataset_dir = args.dataset_dir
        self.generalized_data = args.generalized_data
        self.lr = args.lr
        self.epoch = args.epoch
        self.image_shape = args.image_shape
        self.vector_dims = args.vector_dims
        self.small_network = args.small_network
        self.residual = args.residual
        self.load_checkpoint=args.load
        self.num_textures=8
        if not self.load_checkpoint:
            self.texture_data = nnUtils.import_images_ignore(args.dataset_dir,args.generalized_data,size=[4,args.image_shape[0],args.image_shape[1],args.image_shape[2]])
            self.real_data = nnUtils.import_images(os.path.join(args.dataset_dir,args.generalized_data))
            print("Imported Data!")
            self.num_textures = self.texture_data.shape[0]

        if not self.small_network:
            self.encoder = encoder
            self.decoder = decoder
        else:
            self.encoder = small_network
        self.build()
        if self.load_checkpoint:
            self.load()

build(self) is called within the initialization of Phi and assembles the architecture for the neural network.

In [ ]:
def build(self):
        self.x = tf.placeholder(tf.float32, [None, self.image_shape[0], self.image_shape[1], 3])
        self.y = tf.placeholder(tf.float32, [self.image_shape[0], self.image_shape[1], 3])
        if not self.small_network:
            self.encoder_net = self.encoder(self.x,self.vector_dims, reuse=False)
            self.decoder_net = self.decoder(self.encoder_net,self.image_shape,reuse=False)
        else:
            self.encoder_net = self.encoder(self.x,reuse=False)
        self.vars=tf.trainable_variables()
        if not self.small_network:
            self.loss=loss(self.decoder_net,self.y,self.x,self.num_textures,self.residual)
        else:
            self.loss=loss(self.encoder_net,self.y,self.x,self.num_textures,self.residual)
            
        self.optim = tf.train.AdamOptimizer(self.lr).minimize(self.loss, var_list=self.vars)
        
        #Saver for creating and loading checkpoints
        self.saver = tf.train.Saver()
        print("Built!")


### Instantiate phi object and train 
`with tf.Session(config=tfconfig) as sess:
        model = phi(sess, args)
        model.train(args)`

### Set arguments in phi object
`def __init__(self, sess, args):
        self.sess = sess
        self.dataset_dir_x = args.dataset_dir_x
        self.test_dir_x = args.test_dir_x
        self.fig_output=args.fig_output
        self.epoch = args.epoch
        self.lr = args.lr
        self.dataset_dir_x = args.dataset_dir_x
        self.beta1 = args.beta1
        self.image_shape = args.image_shape
        self.batch_size = args.bs
        self.phi_network = phi_network_residual
        self.input= nnUtils.import_images(self.dataset_dir_x)
        self.input_indecies=np.load(args.indecies_file)
        self.num_bins=args.num_bins
        self.alpha=args.alpha
        self.vector_dims=args.vector_dims
        self.graph = args.graph
        self.graph_freq = args.graph_freq
        self.graph_amount=args.graph_amount
        self.plot_loss=args.plot_loss
        self.solid_shapes = args.solid_shapes
        self.plot3d=args.plot3d
        self.build()`

##### encoder(x,vector_dims) 

###### Purpose: 
Creates an encoder network to create an n-dimensional embedding to represent a feature space of the images

###### args: 

x - the input image to the encoder network

vector_dims - the dimensionality of the output feature space

name - name of the network

activation - the activation function of the 
###### Returns:
A residual layer

In [ ]:
def encoder(x,vector_dims,reuse=False, name="encoder", activation=tf.nn.relu,num_filters=32, kernel_size=[5,5],stride=[1,1]):
    with tf.variable_scope(name, reuse=reuse):
        x = residual_block(x,num_filters,kernel_size)
        x = residual_block(x,num_filters,kernel_size)
        x = residual_block(x,num_filters,kernel_size)
        x = residual_block(x,num_filters,kernel_size)
        x = tf.contrib.slim.conv2d_transpose(x, 1, kernel_size, stride, padding='SAME')
        x = tf.layers.batch_normalization(x)
        x = tf.contrib.layers.flatten(x)
        x = tf.layers.dense(x,vector_dims, activation=activation)
        return x

##### decoder(x,image_shape)

###### Purpose: 
Decode the n-dimensional vector created by the encoder network into an image

###### args: 

x - the input layer

image_shape - the shape of the image to be created

name - name of the layer
###### Returns:
An image

In [ ]:
def decoder(x,image_shape,reuse=False, name="decoder", activation=tf.nn.relu,num_filters=128,stride=[2,2],kernel=[4,4]):
    with tf.variable_scope(name, reuse=reuse):
        x = tf.expand_dims(tf.expand_dims(x, 1), 1)
        print(x.shape)
        x = tf.layers.dense(x,int(image_shape[0]*image_shape[1]*image_shape[2]/16))
        print(x.shape)
        x = tf.reshape(x,[-1,int(image_shape[0]/4),int(image_shape[1]/4),image_shape[2]])
        print(x.shape)
        x = tf.layers.conv2d_transpose(x,num_filters,kernel,stride, padding="SAME")
        print(x.shape)
        x = tf.layers.conv2d_transpose(x,int(num_filters/2),kernel,stride, padding="SAME")
        print(x.shape)
        x = tf.layers.conv2d_transpose(x,int(num_filters/4),kernel,stride, padding="SAME")
        print(x.shape)
        x = tf.layers.conv2d_transpose(x,int(num_filters/8),kernel,stride, padding="SAME")
        print(x.shape)
        x = tf.layers.conv2d_transpose(x,3,kernel,stride, padding="SAME")
        print(x.shape)
        x = tf.layers.conv2d(x,3,kernel,stride, padding="SAME")
        print(x.shape)
        x = tf.layers.conv2d(x,3,kernel,stride, padding="SAME")
        print(x.shape)
        x = tf.layers.conv2d(x,3,kernel,stride, padding="SAME")
        print(x.shape)
        #exit()
        return x

### Train the network

In [ ]:
def train(self,args):
        '''
        Trains the triplet network by having it learn an embedding into args.vector_dims dimensional space.
        :param args: StringArray, Arguments passed in from argument parser
        '''

        loss_scalar = tf.summary.scalar("Loss", self.loss)
        init=tf.global_variables_initializer()
        train_writer = tf.summary.FileWriter( './logs/train ', self.sess.graph)
        dir_path = os.path.dirname(os.path.realpath(__file__))
        checkpoint = os.path.join(dir_path,"checkpoints/",)

        self.sess.run(init)
        iter=0
        print("Training initialized")
        all_loss=[]
        for i in range(self.epoch):
            index_transform = random.randint(0, self.real_data.shape[0]-1)
            texture_batch=np.zeros([self.num_textures,self.image_shape[0],self.image_shape[1],self.image_shape[2]])
            for x in range(self.num_textures):
                texture_batch[x]=self.texture_data[x][index_transform]
            real_batch=self.real_data[index_transform]
            if not self.small_network:
                loss_data, _, generated, _ = self.sess.run([self.loss,self.optim,self.decoder_net, self.encoder_net],feed_dict={self.x: texture_batch, self.y: real_batch})
            else:
                loss_data, _, generated = self.sess.run([self.loss,self.optim,self.encoder_net],feed_dict={self.x: texture_batch, self.y: real_batch})
            #print(loss_data)
            img_texture = 5
            img_texture_2 = 6
            img_texture_3 = 7
            img_mod = 100
            img_path = './results/'
            if i%img_mod==0:
                if self.residual:
                    #Save generated images to path
                    generated_image = util.scale_data(generated[img_texture] + texture_batch[img_texture],[0,255])
                    cv2.imwrite(img_path + 'generated_images/' + str(int(i/img_mod)) + ".jpg", generated_image)
                    generated_image = util.scale_data(generated[img_texture_2] + texture_batch[img_texture_2],[0,255])
                    cv2.imwrite(img_path + 'generated_images_2/' + str(int(i/img_mod)) + ".jpg", generated_image)
                    generated_image = util.scale_data(generated[img_texture_3] + texture_batch[img_texture_3],[0,255])
                    cv2.imwrite(img_path + 'generated_images_3/' + str(int(i/img_mod)) + ".jpg", generated_image)
                    mask = util.scale_data(generated[img_texture],[0,255])
                    cv2.imwrite(img_path + 'masks/' + str(int(i/img_mod)) + ".jpg", mask)
                    mask = util.scale_data(generated[img_texture_2],[0,255])
                    cv2.imwrite(img_path + 'masks_2/' + str(int(i/img_mod)) + ".jpg", mask)
                    mask = util.scale_data(generated[img_texture_3],[0,255])
                    cv2.imwrite(img_path + 'masks_3/' + str(int(i/img_mod)) + ".jpg", mask)
                else:
                    img = util.scale_data(generated[img_texture],[0,255])
                    cv2.imwrite(img_path + 'generated_images/' + str(int(i/img_mod)) + ".jpg", img)
                    img = util.scale_data(generated[img_texture_2],[0,255])
                    cv2.imwrite(img_path + 'generated_images_2/' + str(int(i/img_mod)) + ".jpg", img)
                    img = util.scale_data(generated[img_texture_3],[0,255])
                    cv2.imwrite(img_path + 'generated_images_3/' + str(int(i/img_mod)) + ".jpg", img)
                target_image = util.scale_data(real_batch,[0,255])
                cv2.imwrite(img_path + 'target/' + str(int(i/img_mod)) + ".jpg", target_image)
                original_image = util.scale_data(texture_batch[img_texture],[0,255])
                cv2.imwrite(img_path + 'original/' + str(int(i/img_mod)) + ".jpg", original_image)
                original_image = util.scale_data(texture_batch[img_texture_2],[0,255])
                cv2.imwrite(img_path + 'original_2/' + str(int(i/img_mod)) + ".jpg", original_image)
                original_image = util.scale_data(texture_batch[img_texture_3],[0,255])
                cv2.imwrite(img_path + 'original_3/' + str(int(i/img_mod)) + ".jpg", original_image)
                self.saver.save(self.sess, checkpoint, global_step=img_mod)


### Load checkpoint to resume training

In [ ]:
    def load(self):
        dir_path = os.path.dirname(os.path.realpath(__file__))
        checkpoint = os.path.join(dir_path,"checkpoints/",)
        self.saver.restore(self.sess, tf.train.latest_checkpoint(checkpoint))